# Getting Google Form Data to Dataframe Using API

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pandas as pd
import schedule
import time
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
import smtplib
import os

In [ ]:
# Initializing a GoogleAuth Object
gauth = GoogleAuth()

In [ ]:
# client_secrets.json file is verified
# and it automatically handles authentication
gauth.LocalWebserverAuth()

In [ ]:
# GoogleDrive Instance is created using
# authenticated GoogleAuth instance
drive = GoogleDrive(gauth)

In [ ]:
# Initialize GoogleDriveFile instance with file id
file_obj = drive.CreateFile({'id': '1otxIO1cdOkJ1ks-QvvING8z8p7uypaX0xKtYVjT4ZcM'})

In [ ]:
file_obj.GetContentFile('Profile (Responses).xls',
         mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')

In [ ]:
dataframe = pd.read_excel('Profile (Responses).xls')

In [ ]:
print(dataframe)

## Sending Mailto

In [ ]:
Mailto = dataframe.loc[1,'Email ID']

# Sending Mail at Regular Time Interval

In [ ]:
# send our email message 'msg' to our boss
def message(subject="Python Notification",
            text="", img=None, attachment=None):
    # build message contents
    msg = MIMEMultipart()

    # Add Subject
    msg['Subject'] = subject
    
    # Add text contents
    msg.attach(MIMEText(text))

    # Check if we have anything
    # given in the img parameter
    if img is not None:

        # Check whether we have the
        # lists of images or not!
        if type(img) is not list:
            # if it isn't a list, make it one
            img = [img]

        # Now iterate through our list
        for one_img in img:

            # read the image binary data
            img_data = open(one_img, 'rb').read()

            # Attach the image data to MIMEMultipart
            # using MIMEImage,
            # we add the given filename use os.basename
            msg.attach(MIMEImage(img_data,
                                 name=os.path.basename(one_img)))

    # We do the same for attachments
    # as we did for images
    if attachment is not None:

    # Check whether we have the
    # lists of attachments or not!
        if type(attachment) is not list:

            # if it isn't a list, make it one
            attachment = [attachment]

        for one_attachment in attachment:

            with open(one_attachment, 'rb') as f:

                # Read in the attachment using MIMEApplication
                file = MIMEApplication(
                    f.read(),
                    name=os.path.basename(one_attachment)
                )
            file['Content-Disposition'] = f'attachment;\
            filename="{os.path.basename(one_attachment)}"'

            # At last, Add the attachment to our message object
            msg.attach(file)
    return msg


def mail():

    # initialize connection to our email server,
    # we will use gmail here
    smtp = smtplib.SMTP('smtp.gmail.com', 587)
    smtp.ehlo()
    smtp.starttls()
    
    # Login with your email and password
    smtp.login('shindetanmay73@gmail.com', 'qfrtxhjsjpaphfei')

    # Call the message function
    msg = message("Good!", "Hi there!",
                r"C:\Users\shrey\OneDrive\Pictures\TESLA.png")

    # Make a list of emails, where you wanna send mail
    to = [Mailto]

    # Provide some data to the sendmail function!
    smtp.sendmail(from_addr="shindetanmay73@gmail.com",
                to_addrs=to, msg=msg.as_string())

    # Finally, don't forget to close the connection
    smtp.quit()


schedule.every(2).seconds.do(mail)
#schedule.every(10).minutes.do(mail)
#schedule.every().hour.do(mail)
#schedule.every().day.at("10:30").do(mail)
#schedule.every(5).to(10).minutes.do(mail)
#schedule.every().monday.do(mail)
#schedule.every().wednesday.at("13:15").do(mail)
#schedule.every().minute.at(":17").do(mail)

while True:
    schedule.run_pending()
    time.sleep(1)